In [26]:
from pymongo import MongoClient, ASCENDING, DESCENDING
from datetime import datetime
from bson.objectid import ObjectId

# Подключение к MongoDB
client = MongoClient('mongodb://localhost:27017/')
print("Версия сервера MongoDB:", client.server_info()["version"])
db = client['equipment_management']
print(client.list_database_names())

Версия сервера MongoDB: 3.6.8
['admin', 'config', 'equipment_management', 'local', 'mydb']


In [13]:
# Создаем валидацию для коллекции equipment
db.command({
    'collMod': 'equipment',
    'validator': {
        '$jsonSchema': {
            'bsonType': 'object',
            'required': ['name', 'category', 'purchase_date', 'status'],
            'properties': {
                'status': {
                    'enum': ['на складе', 'в работе', 'в ремонте', 'изъято'],
                    'description': "Статус должен быть одним из: 'на складе', 'в работе', 'в ремонте', 'изъято'"
                }
            }
        }
    }
})

{'ok': 1.0}

In [14]:
# Коллекции
equipment = db['equipment']
employees = db['employees']

# Очистка коллекций перед добавлением тестовых данных
equipment.delete_many({})
employees.delete_many({})

# Добавление тестовых работников
employee1 = employees.insert_one({
    'full_name': 'Иванов Иван Иванович',
    'hire_date': datetime(2020, 5, 15),
    'position': 'Старший механик',
    'assigned_equipment': []
}).inserted_id

employee2 = employees.insert_one({
    'full_name': 'Петров Петр Петрович',
    'hire_date': datetime(2021, 3, 10),
    'position': 'Электроник',
    'assigned_equipment': []
}).inserted_id

employee3 = employees.insert_one({
    'full_name': 'Сидорова Анна Михайловна',
    'hire_date': datetime(2022, 1, 20),
    'position': 'Техник',
    'assigned_equipment': []
}).inserted_id

# Добавление тестового оборудования
equipment_data = [
    {
        'name': 'Токарный станок X-200',
        'category': 'механика',
        'purchase_date': datetime(2019, 10, 5),
        'status': 'в работе',
        'assigned_employee': None
    },
    {
        'name': 'Осциллограф DS-1054',
        'category': 'электроника',
        'purchase_date': datetime(2020, 2, 15),
        'status': 'в ремонте',
        'assigned_employee': employee1
    },
    {
        'name': 'Компрессор AIR-300',
        'category': 'пневматика',
        'purchase_date': datetime(2021, 5, 20),
        'status': 'на складе',
        'assigned_employee': None
    },
    {
        'name': 'Паяльная станция PS-90',
        'category': 'электроника',
        'purchase_date': datetime(2021, 7, 12),
        'status': 'в ремонте',
        'assigned_employee': employee2
    },
    {
        'name': 'Гидравлический пресс HP-50',
        'category': 'гидравлика',
        'purchase_date': datetime(2022, 1, 8),
        'status': 'в работе',
        'assigned_employee': None
    },
    {
        'name': 'Мультиметр UT-61',
        'category': 'электроника',
        'purchase_date': datetime(2022, 3, 10),
        'status': 'изъято',
        'assigned_employee': None
    },
    {
        'name': 'Фрезерный станок F-100',
        'category': 'механика',
        'purchase_date': datetime(2022, 4, 5),
        'status': 'на складе',
        'assigned_employee': None
    }
]

In [15]:
# Вставка оборудования и обновление списков оборудования у работников
inserted_equipment = equipment.insert_many(equipment_data)
equipment_ids = inserted_equipment.inserted_ids

# Назначение оборудования работникам
employees.update_one(
    {'_id': employee1},
    {'$push': {'assigned_equipment': equipment_ids[1]}}
)

employees.update_one(
    {'_id': employee2},
    {'$push': {'assigned_equipment': equipment_ids[3]}}
)

# Функции для работы с базой данных
def search_equipment(query):
    """Поиск оборудования по названию или категории"""
    return list(equipment.find({
        '$or': [
            {'name': {'$regex': query, '$options': 'i'}},
            {'category': {'$regex': query, '$options': 'i'}}
        ]
    }))

In [16]:
def get_equipment_by_status(status):
    """Получение оборудования по статусу"""
    return list(equipment.find({'status': status}))

def get_employees_with_equipment():
    """Получение работников с назначенным оборудованием"""
    return list(employees.find({'assigned_equipment.0': {'$exists': True}}))

def sort_equipment_by_date(direction=ASCENDING):
    """Сортировка оборудования по дате поступления"""
    return list(equipment.find().sort('purchase_date', direction))

In [17]:
def group_equipment_by_category():
    """Группировка оборудования по категориям"""
    pipeline = [
        {'$group': {
            '_id': '$category',
            'count': {'$sum': 1},
            'equipment': {'$push': '$name'}
        }},
        {'$sort': {'count': -1}}
    ]
    return list(equipment.aggregate(pipeline))

In [19]:
# Функция для проекции
def get_equipment_with_projection(fields):
    """Получение оборудования с указанными полями"""
    pipeline = [
        {'$project': fields}
    ]
    return list(equipment.aggregate(pipeline))

In [22]:
# Функция для объединения коллекций ($lookup)
def get_equipment_with_employees():
    """Получение оборудования с информацией о назначенных работниках"""
    pipeline = [
        {
            '$lookup': {
                'from': 'employees',
                'localField': 'assigned_employee',
                'foreignField': '_id',
                'as': 'employee_info'
            }
        },
        {
            '$unwind': {
                'path': '$employee_info',
                'preserveNullAndEmptyArrays': True
            }
        },
        {
            '$project': {
                'name': 1,
                'category': 1,
                'status': 1,
                'employee_name': '$employee_info.full_name',
                'employee_position': '$employee_info.position'
            }
        }
    ]
    return list(equipment.aggregate(pipeline))

In [27]:
# Примеры использования
print("Оборудование в ремонте:")
for item in get_equipment_by_status('в ремонте'):
    print(f"- {item['name']} (Категория: {item['category']})")

print("\nРаботники с назначенным оборудованием:")
for emp in get_employees_with_equipment():
    print(f"- {emp['full_name']} ({emp['position']})")

print("\nОборудование, отсортированное по дате поступления (новые сначала):")
for item in sort_equipment_by_date(DESCENDING):
    print(f"- {item['name']} ({item['purchase_date'].strftime('%Y-%m-%d')})")

print("\nГруппировка оборудования по категориям:")
for group in group_equipment_by_category():
    print(f"{group['_id']}: {group['count']} единиц")

print("\nПоиск оборудования по запросу 'станок':")
for item in search_equipment('станок'):
    print(f"- {item['name']} ({item['category']})")
    
print("\nПроекция (только name и status):")
for item in get_equipment_with_projection({'_id': 0, 'name': 1, 'status': 1}):
    print(item)
print("\nОборудование с информацией о работниках:")
for item in get_equipment_with_employees():
    print(f"{item['name']} ({item['status']}) - Работник: {item.get('employee_name', 'не назначен')}")

Оборудование в ремонте:
- Токарный станок X-200 (Категория: механика)
- Осциллограф DS-1054 (Категория: электроника)
- Паяльная станция PS-90 (Категория: электроника)

Работники с назначенным оборудованием:
- Михнева Анна Денисовна (Менеджер)

Оборудование, отсортированное по дате поступления (новые сначала):
- Фрезерный станок F-100 (2022-04-05)
- Мультиметр UT-61 (2022-03-10)
- Гидравлический пресс HP-50 (2022-01-08)
- Паяльная станция PS-90 (2021-07-12)
- Компрессор AIR-300 (2021-05-20)
- Осциллограф DS-1054 (2020-02-15)
- Токарный станок X-200 (2019-10-05)

Группировка оборудования по категориям:
электроника: 3 единиц
механика: 2 единиц
гидравлика: 1 единиц
пневматика: 1 единиц

Поиск оборудования по запросу 'станок':
- Токарный станок X-200 (механика)
- Фрезерный станок F-100 (механика)

Проекция (только name и status):
{'name': 'Токарный станок X-200', 'status': 'в ремонте'}
{'name': 'Осциллограф DS-1054', 'status': 'в ремонте'}
{'name': 'Компрессор AIR-300', 'status': 'на складе

Фишка базы данных

Мне особенно понравилась возможность MongoDB выполнять сложные агрегации данных с помощью pipeline. В этом проекте я использовал агрегацию для группировки оборудования по категориям с подсчетом количества в каждой категории. Pipeline в MongoDB позволяет строить сложные цепочки обработки данных, включая:

    Фильтрацию ($match)

    Группировку ($group)

    Сортировку ($sort)

    Проекции ($project)

    Объединение с другими коллекциями ($lookup)

Это мощный инструмент для анализа данных без необходимости писать сложный код на стороне приложения. В реляционных базах подобные операции часто требуют сложных JOIN-запросов, тогда как в MongoDB они выполняются более интуитивно.

Также мне понравилась гибкость схемы документов - разные категории оборудования могут иметь разные атрибуты, но храниться в одной коллекции. Это упрощает расширение системы в будущем при появлении новых типов оборудования.